In [274]:
import numpy as np
import plotly
import pandas as pd
import os
import plotly.offline as pyo
import plotly.express as px
import plotly.io as pio
import plotly.graph_objects as go
from pandas.plotting import scatter_matrix
import plotly.express as px
import json
import functions as f

import warnings
warnings.filterwarnings('ignore')

PATHS = {
    'dataset_1': './datasets/Dataset1.- DatosConsumoAlimentarioMAPAporCCAA.txt',
    'dataset_2': './datasets/Dataset2.- Precios Semanales Observatorio de Precios Junta de Andalucia.txt',
    'dataset_3': './datasets/Dataset3a_Datos_MercaMadrid.txt',
    'dataset_4': './datasets/Dataset4.- Comercio Exterior de España.txt',
    'dataset_5': './datasets/Dataset5_Coronavirus_cases.txt'
}

MES = {
    'Enero': 1,
    'Febrero': 2, 
    'Marzo': 3, 
    'Abril': 4, 
    'Mayo': 5, 
    'Junio': 6, 
    'Julio': 7,
    'Agosto': 8, 
    'Septiembre': 9, 
    'Octubre': 10, 
    'Noviembre': 11, 
    'Diciembre': 12
}

NUM2MES = {b:a for a,b in zip(MES.keys(), MES.values())}

TITULOS = {
    'Año': 'Any',
    'Mes': 'mes',
    'CCAA': 'CCAA',
    'Producto': 'producto',
    'Volumen (miles de kg)': 'volumen',
    'Valor (miles de €)': 'valor', 
    'Precio medio kg': 'precio', 
    'Penetración (%)': 'penetracion',
    'Consumo per capita':'consumo_per_capita', 
    'Gasto per capita': 'gasto_per_capita'
}

norm = {
    'Andalucía': 'Andalucia',
    'Aragón': 'Aragon',
    'Cantabria': 'Cantabria',
    'Castilla y León': 'Castilla Leon',
    'Castilla-La Mancha': 'Castilla La Mancha',
    'Cataluña': 'Cataluña',
    'Islas Baleares': 'Baleares',
    'Islas Canarias': 'Canarias',
    'La Rioja': 'La Rioja',
    'País Vasco': 'País Vasco',
    'Principado de Asturias': 'Asturias',
    'Región de Murcia': 'Murcia',
    'Galicia': 'Galicia',
    'Ceuta y Melilla': 'Andalucia', # Lo mas cercano
    'Comunidad de Madrid': 'Madrid',
    'Comunidad Foral de Navarra': 'Navarra',
    'Comunidad Valenciana': 'Valencia',
    'Extremadura': 'Extremadura'
}

norm_id = {'Andalucia': 'Andalucia',
         'Aragon': 'Aragon',
         'Asturias': 'Asturias',
         'Baleares': 'Baleares',
         'Canarias': 'Canarias',
         'Cantabria': 'Cantabria',
         'Castilla La Mancha': 'Castilla La Mancha',
         'Castilla Leon': 'Castilla Leon',
         'Cataluña': 'Cataluña',
         'Extremadura': 'Extremadura',
         'Galicia': 'Galicia',
         'La Rioja': 'La Rioja',
         'Madrid': 'Madrid',
         'Murcia': 'Murcia',
         'Navarra': 'Navarra',
         'Pais Vasco': 'Pais Vasco',
         'Valencia': 'Valencia',
         'Total Nacional': 'Spain'
}


In [197]:
# /////////////////////////////////////
# EDA Dataset 1
# ///////////////////////////////////// 
df = pd.read_csv(PATHS['dataset_1'], sep= '|', decimal=",", index_col = None)\
        .drop(columns= ['Unnamed: 10', 'Unnamed: 11'])     \
        .rename(columns= TITULOS)
# Adding numeric month
df['mes_n'] = df['mes'].apply(lambda mes: MES[mes])
df = df[['Any', 'mes', 'mes_n', 'CCAA', 'producto', 'volumen', 'valor', 'precio',
       'penetracion', 'consumo_per_capita', 'gasto_per_capita']]
df['id'] = df['CCAA'].apply(lambda r: norm_id[r])
df.head(3)

,Any,mes,mes_n,CCAA,producto,volumen,valor,precio,penetracion,consumo_per_capita,gasto_per_capita,id
0,2018,Enero,1,Total Nacional,TOTAL PATATAS,108430.72,84640.08,0.78,79.40,2.38,1.85,Spain
1,2018,Enero,1,Total Nacional,PATATAS FRESCAS,79445.66,54688.29,0.69,68.46,1.74,1.20,Spain
2,2018,Enero,1,Total Nacional,PATATAS CONGELADAS,3999.90,4857.79,1.21,12.06,0.09,0.11,Spain


<h1> 1- Cuales son los productos mas caros por orden de meses?

In [296]:
# Eliminar todas las Comunidades Autonomas
#df = df.drop('CCAA', axis= 1)

# Agrupamos pos productos
df1 = pd.DataFrame(df.groupby(['Any','mes_n','producto'])[['Any','producto','mes_n','precio', 'valor', 'penetracion', 'consumo_per_capita', 'gasto_per_capita']].mean())\
            .sort_values(by='precio', ascending= False).head(5).reset_index(drop=True)
df1
#df.head()
# for y, m in sorted(set(zip(df1.Any, df1.mes_n))):
#     print(f'{y:>20}/{m}')
#     data = df1[(df1.Any == y) & (df1.mes_n == m) & (df1.CCAA != 'Total Nacional')].sort_values(by= 'precio', ascending= False).head()
#     t = data.keys()
#     print(f'{t[0]:^6}|{t[1]:^10}|{t[3]:^15}|{t[4]:^18}|{t[7]:^8}|{t[6]:^8}|{t[5]:^8}|{t[8]:^13}|{t[9]:^15}\n{"-"*110}')
#     for i in range(min(5, data.mes.shape[0])):
#         r = data.iloc[i]
#         print(f'{r[0]:^6}|{r[1]:^10}|{r[3]:^15}|{r[4]:^18}|{r[7]:^8}|{r[6]:^8}|{r[5]:^8}|{r[8]:^13}|{r[9]:^15}')
#     print()

,Any,mes_n,precio,valor,penetracion,consumo_per_capita,gasto_per_capita
0,2020,4,6.678333,4894.586667,51.096111,0.140000,0.926667
1,2020,3,6.636667,3865.445000,47.174444,0.112222,0.737778
2,2020,5,6.590556,4311.426111,48.998333,0.121667,0.798889
3,2020,6,6.553333,3537.686667,47.016667,0.100556,0.654444
4,2019,3,6.488889,2983.161111,40.021111,0.080556,0.505000


In [234]:
for y, m in sorted(set(zip(df.Any, df.mes_n))):
    print(f'{y:>20}/{m}')
    data = df[(df.Any == y) & (df.mes_n == m) & (df.CCAA != 'Total Nacional')
             ].sort_values(by= 'precio', ascending= False).head()
    t = data.keys()
    print(f'{t[0]:^6}|{t[1]:^10}|{t[3]:^15}|{t[4]:^18}|{t[7]:^8}|{t[6]:^8}|{t[5]:^8}|{t[8]:^13}|{t[9]:^15}\n{"-"*110}')
    for i in range(min(5, data.mes.shape[0])):
        r = data.iloc[i]
        print(f'{r[0]:^6}|{r[1]:^10}|{r[3]:^15}|{r[4]:^18}|{r[7]:^8}|{r[6]:^8}|{r[5]:^8}|{r[8]:^13}|{r[9]:^15}')
    print()

                2018/1
 Any  |   mes    |     CCAA      |     producto     | precio | valor  |volumen | penetracion |consumo_per_capita
--------------------------------------------------------------------------------------------------------------
 2018 |  Enero   |    Navarra    |     CEREZAS      | 21.91  | 14.31  |  0.65  |    1.06     |      0.0      
 2018 |  Enero   |   Canarias    |     CEREZAS      | 10.73  | 37.58  |  3.5   |    1.27     |      0.0      
 2018 |  Enero   |   Cataluña    |     CEREZAS      |  9.22  | 119.02 | 12.91  |    1.08     |      0.0      
 2018 |  Enero   |   Cantabria   |    ESPARRAGOS    |  8.33  |  4.76  |  0.57  |    0.74     |      0.0      
 2018 |  Enero   | Castilla Leon |     CEREZAS      |  7.35  |  79.2  | 10.77  |    0.56     |      0.0      

                2018/2
 Any  |   mes    |     CCAA      |     producto     | precio | valor  |volumen | penetracion |consumo_per_capita
------------------------------------------------------------------

 Any  |   mes    |     CCAA      |     producto     | precio | valor  |volumen | penetracion |consumo_per_capita
--------------------------------------------------------------------------------------------------------------

                2020/10
 Any  |   mes    |     CCAA      |     producto     | precio | valor  |volumen | penetracion |consumo_per_capita
--------------------------------------------------------------------------------------------------------------

                2020/11
 Any  |   mes    |     CCAA      |     producto     | precio | valor  |volumen | penetracion |consumo_per_capita
--------------------------------------------------------------------------------------------------------------



<h1> 2- Cuales son los productos de mas volumen por orden de meses?

In [141]:
df.producto.unique()

array(['TOTAL PATATAS', 'PATATAS FRESCAS', 'PATATAS CONGELADAS',
       'PATATAS PROCESADAS', 'T.HORTALIZAS FRESCAS', 'TOMATES',
       'CEBOLLAS', 'AJOS', 'COLES', 'PEPINOS', 'JUDIAS VERDES',
       'PIMIENTOS', 'CHAMPIÑONES+O.SETAS', 'LECHUGA/ESC./ENDIVIA',
       'ESPARRAGOS', 'VERDURAS DE HOJA', 'BERENJENAS', 'ZANAHORIAS',
       'CALABACINES', 'OTR.HORTALIZAS/VERD.', 'BROCOLI', 'ALCACHOFAS',
       'VERD./HORT. IV GAMA', 'T.FRUTAS FRESCAS', 'NARANJAS',
       'MANDARINAS', 'LIMONES', 'PLATANOS', 'MANZANAS', 'PERAS',
       'MELOCOTONES', 'NECTARINAS', 'ALBARICOQUES', 'FRESAS/FRESON',
       'MELON', 'SANDIA', 'CIRUELAS', 'CEREZAS', 'UVAS', 'KIWI',
       'AGUACATE', 'PIÑA', 'OTRAS FRUTAS FRESCAS', 'POMELO', 'CHIRIMOYA',
       'FRUTAS IV GAMA', 'PATATAS FRITAS', 'APIO', 'COLIFLOR', 'PUERRO',
       'MANGO'], dtype=object)

In [143]:
for y, m in sorted(set(zip(df.Any, df.mes_n))):
    print(f'{y:>20}/{m}')
    data = df[(df.Any == y) & (df.mes_n == m) & #(df.CCAA != 'Total Nacional') &
             (df.producto != 'T.FRUTAS FRESCAS') & (df.producto != 'T.HORTALIZAS FRESCAS') &
             (df.producto != 'TOTAL PATATAS')].sort_values(by= 'volumen', ascending= False).head()
    t = data.keys()
    print(f'{t[0]:^6}|{t[1]:^10}|{t[3]:^15}|{t[4]:^18}|{t[7]:^8}|{t[6]:^8}|{t[5]:^8}|{t[8]:^13}|{t[9]:^15}\n{"-"*110}')
    for i in range(min(5, data.mes.shape[0])):
        r = data.iloc[i]
        print(f'{r[0]:^6}|{r[1]:^10}|{r[3]:^15}|{r[4]:^18}|{r[7]:^8}|{r[6]:^8}|{r[5]:^8}|{r[8]:^13}|{r[9]:^15}')
    print()

                2018/1
 Any  |   mes    |   producto    |     volumen      |penetracion| precio | valor  |consumo_per_capita|gasto_per_capita
--------------------------------------------------------------------------------------------------------------
 2018 |  Enero   |   NARANJAS    |     96414.9      | 60.45  |  0.92  |88258.06|    2.11     |     1.93      
 2018 |  Enero   |PATATAS FRESCAS|     79445.66     | 68.46  |  0.69  |54688.29|    1.74     |      1.2      
 2018 |  Enero   |   PLATANOS    |     51819.21     | 80.36  |  1.45  |74986.61|    1.14     |     1.64      
 2018 |  Enero   |OTR.HORTALIZAS/VERD.|     50041.09     | 82.31  |  1.93  |96616.8 |     1.1     |     2.12      
 2018 |  Enero   |  MANDARINAS   |     46857.48     | 62.46  |  1.35  |63487.81|    1.03     |     1.39      

                2018/2
 Any  |   mes    |   producto    |     volumen      |penetracion| precio | valor  |consumo_per_capita|gasto_per_capita
-------------------------------------------------

 Any  |   mes    |   producto    |     volumen      |penetracion| precio | valor  |consumo_per_capita|gasto_per_capita
--------------------------------------------------------------------------------------------------------------
 2019 | Octubre  |PATATAS FRESCAS|     80084.23     | 66.79  |  0.88  |70668.9 |    1.74     |     1.53      
 2019 | Octubre  |    TOMATES    |     54632.81     | 78.05  |  1.75  |95707.3 |    1.18     |     2.08      
 2019 | Octubre  |   PLATANOS    |     48788.85     |  78.0  |  1.76  |85747.1 |    1.06     |     1.86      
 2019 | Octubre  |   MANZANAS    |     44647.66     |  65.1  |  1.46  |65402.31|    0.97     |     1.42      
 2019 | Octubre  |OTR.HORTALIZAS/VERD.|     36381.1      | 75.91  |  2.28  |82817.2 |    0.79     |      1.8      

                2019/11
 Any  |   mes    |   producto    |     volumen      |penetracion| precio | valor  |consumo_per_capita|gasto_per_capita
-----------------------------------------------------------------------

<h1>3- De que es el mayor consumo per capita por orden de meses?

In [138]:
for y, m in sorted(set(zip(df.Any, df.mes_n))):
    print(f'{y:>20}/{m}')
    data = df[(df.Any == y) & (df.mes_n == m) & (df.CCAA != 'Total Nacional') &
             (df.producto != 'T.FRUTAS FRESCAS') & (df.producto != 'T.HORTALIZAS FRESCAS') &
             (df.producto != 'TOTAL PATATAS')].sort_values(by= 'consumo_per_capita', ascending= False).head()
    t = data.keys()
    print(f'{t[0]:^6}|{t[1]:^10}|{t[3]:^15}|{t[4]:^18}|{t[7]:^8}|{t[6]:^8}|{t[5]:^8}|{t[8]:^13}|{t[9]:^15}\n{"-"*110}')
    for i in range(min(5, data.mes.shape[0])):
        r = data.iloc[i]
        print(f'{r[0]:^6}|{r[1]:^10}|{r[3]:^15}|{r[4]:^18}|{r[7]:^8}|{r[6]:^8}|{r[5]:^8}|{r[8]:^13}|{r[9]:^15}')
    print()

                2018/1
 Any  |   mes    |     CCAA      |     producto     | precio | valor  |volumen | penetracion |consumo_per_capita
--------------------------------------------------------------------------------------------------------------
 2018 |  Enero   |   Cantabria   |     NARANJAS     |  0.96  |1580.62 | 1646.3 |    66.43    |     2.96      
 2018 |  Enero   |  Extremadura  |     NARANJAS     |  0.89  |2889.41 |3243.68 |    51.7     |     2.75      
 2018 |  Enero   |  Pais Vasco   |     NARANJAS     |  0.98  |5264.45 |5372.93 |    66.66    |      2.6      
 2018 |  Enero   | Castilla Leon |     NARANJAS     |  0.87  |5496.28 |6338.92 |    68.58    |     2.59      
 2018 |  Enero   |    Galicia    |     NARANJAS     |  0.94  |6573.02 |7029.69 |    68.28    |     2.54      

                2018/2
 Any  |   mes    |     CCAA      |     producto     | precio | valor  |volumen | penetracion |consumo_per_capita
------------------------------------------------------------------

 Any  |   mes    |     CCAA      |     producto     | precio | valor  |volumen | penetracion |consumo_per_capita
--------------------------------------------------------------------------------------------------------------
 2019 |  Julio   |    Navarra    | PATATAS FRESCAS  |  0.93  |2416.63 | 2608.9 |    66.87    |     4.36      
 2019 |  Julio   |    Galicia    | PATATAS FRESCAS  |  0.84  |8939.29 |10684.33|    57.49    |     3.93      
 2019 |  Julio   |  Extremadura  |     TOMATES      |  1.65  |5799.45 |3506.21 |    86.74    |     2.98      
 2019 |  Julio   |    Navarra    |     CEBOLLAS     |  0.92  |1614.51 |1759.91 |    71.75    |     2.94      
 2019 |  Julio   |   Baleares    |      SANDIA      |  0.75  |2222.17 |2974.56 |    73.89    |     2.82      

                2019/8
 Any  |   mes    |     CCAA      |     producto     | precio | valor  |volumen | penetracion |consumo_per_capita
-----------------------------------------------------------------------------------------

<h1>4- Cuales es el mayor gasto per capita?

In [200]:
for y, m in sorted(set(zip(df.Any, df.mes_n))):
    print(f'{y:>20}/{m}')
    data = df[(df.Any == y) & (df.mes_n == m) & (df.CCAA != 'Total Nacional') &
             (df.producto != 'T.FRUTAS FRESCAS') & (df.producto != 'T.HORTALIZAS FRESCAS') &
             (df.producto != 'TOTAL PATATAS')].sort_values(by= 'gasto_per_capita', ascending= False).head()
    t = data.keys()
    print(f'{t[0]:^6}|{t[1]:^10}|{t[3]:^15}|{t[4]:^18}|{t[7]:^8}|{t[6]:^8}|{t[5]:^8}|{t[8]:^13}|{t[9]:^15}|{t[10]:^15}\n{"-"*110}')
    for i in range(min(5, data.mes.shape[0])):
        r = data.iloc[i]
        print(f'{r[0]:^6}|{r[1]:^10}|{r[3]:^15}|{r[4]:^18}|{r[7]:^8}|{r[6]:^8}|{r[5]:^8}|{r[8]:^13}|{r[9]:^15}|{r[10]:^15}')
    print()

                2018/1
 Any  |   mes    |     CCAA      |     producto     | precio | valor  |volumen | penetracion |consumo_per_capita|gasto_per_capita
--------------------------------------------------------------------------------------------------------------
 2018 |  Enero   |   Cataluña    |OTR.HORTALIZAS/VERD.|  1.88  |20661.43|10982.28|    88.51    |     1.59      |     2.99      
 2018 |  Enero   |   Cantabria   |     NARANJAS     |  0.96  |1580.62 | 1646.3 |    66.43    |     2.96      |     2.84      
 2018 |  Enero   |   Valencia    |OTR.HORTALIZAS/VERD.|  1.81  |13072.09|7230.29 |    86.63    |     1.53      |     2.76      
 2018 |  Enero   |  Pais Vasco   |     NARANJAS     |  0.98  |5264.45 |5372.93 |    66.66    |      2.6      |     2.54      
 2018 |  Enero   |   Canarias    |OTR.HORTALIZAS/VERD.|  2.5   |5463.02 |2182.63 |    88.32    |      1.0      |      2.5      

                2018/2
 Any  |   mes    |     CCAA      |     producto     | precio | valor  |volum

 Any  |   mes    |     CCAA      |     producto     | precio | valor  |volumen | penetracion |consumo_per_capita|gasto_per_capita
--------------------------------------------------------------------------------------------------------------
 2019 |  Julio   |  Extremadura  |     TOMATES      |  1.65  |5799.45 |3506.21 |    86.74    |     2.98      |     4.93      
 2019 |  Julio   |    Navarra    | PATATAS FRESCAS  |  0.93  |2416.63 | 2608.9 |    66.87    |     4.36      |     4.04      
 2019 |  Julio   |    Navarra    |LECHUGA/ESC./ENDIVIA|  2.3   |2075.24 | 901.89 |    68.52    |     1.51      |     3.47      
 2019 |  Julio   |    Galicia    | PATATAS FRESCAS  |  0.84  |8939.29 |10684.33|    57.49    |     3.93      |     3.29      
 2019 |  Julio   |   Cataluña    |     TOMATES      |  1.71  |22954.12|13401.91|    87.62    |     1.87      |     3.21      

                2019/8
 Any  |   mes    |     CCAA      |     producto     | precio | valor  |volumen | penetracion |consumo_p

<h1> 5- Relaciones Volumen-Precio

In [202]:
df.head()

,Any,mes,mes_n,CCAA,producto,volumen,valor,precio,penetracion,consumo_per_capita,gasto_per_capita,id
0,2018,Enero,1,Total Nacional,TOTAL PATATAS,108430.72,84640.08,0.78,79.40,2.38,1.85,Spain
1,2018,Enero,1,Total Nacional,PATATAS FRESCAS,79445.66,54688.29,0.69,68.46,1.74,1.20,Spain
2,2018,Enero,1,Total Nacional,PATATAS CONGELADAS,3999.90,4857.79,1.21,12.06,0.09,0.11,Spain
3,2018,Enero,1,Total Nacional,PATATAS PROCESADAS,4997.03,25094.00,5.02,45.94,0.11,0.55,Spain
4,2018,Enero,1,Total Nacional,T.HORTALIZAS FRESCAS,209957.24,376688.56,1.79,97.27,4.60,8.25,Spain


In [247]:
df2 = df
df2['volXprecio'] = df2.apply(lambda x: x.volumen * x.precio, axis= 1)
df2['consumo/gasto'] = df2.apply(lambda x: x.consumo_per_capita / x.gasto_per_capita if x.gasto_per_capita else 0, axis= 1)
df2

,Any,mes,mes_n,CCAA,producto,volumen,valor,precio,penetracion,consumo_per_capita,gasto_per_capita,id,volXprecio,consumo/gasto
0,2018,Enero,1,Total Nacional,TOTAL PATATAS,108430.72,84640.08,0.78,79.40,2.38,1.85,Spain,84575.9616,1.286486
1,2018,Enero,1,Total Nacional,PATATAS FRESCAS,79445.66,54688.29,0.69,68.46,1.74,1.20,Spain,54817.5054,1.450000
2,2018,Enero,1,Total Nacional,PATATAS CONGELADAS,3999.90,4857.79,1.21,12.06,0.09,0.11,Spain,4839.8790,0.818182
3,2018,Enero,1,Total Nacional,PATATAS PROCESADAS,4997.03,25094.00,5.02,45.94,0.11,0.55,Spain,25085.0906,0.200000
4,2018,Enero,1,Total Nacional,T.HORTALIZAS FRESCAS,209957.24,376688.56,1.79,97.27,4.60,8.25,Spain,375823.4596,0.557576
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26629,2020,Noviembre,11,Total Nacional,UVAS,15918.49,42755.62,2.69,NaN,0.36,0.96,Spain,42820.7381,0.375000
26630,2020,Noviembre,11,Total Nacional,KIWI,12253.15,41949.06,3.42,NaN,0.28,0.95,Spain,41905.7730,0.294737
26631,2020,Noviembre,11,Total Nacional,PATATAS FRESCAS,89679.98,76899.34,0.86,NaN,2.02,1.73,Spain,77124.7828,1.167630
26632,2020,Noviembre,11,Total Nacional,T.HORTALIZAS FRESCAS,238554.47,472592.86,1.98,NaN,5.38,10.66,Spain,472337.8506,0.504690


In [333]:
def porAny(a, d):
    df = d.copy()
    df3 = df[df.Any == a]
    df3.groupby(['producto'])[['valor', 'precio', 'volumen', 'penetracion', 'volXprecio', 'consumo/gasto','gasto_per_capita','consumo_per_capita']].mean()
    df3['any'] = a
    return df3[['producto', 'any', 'valor', 'precio', 'volumen', 'penetracion', 'volXprecio', 'consumo/gasto','gasto_per_capita','consumo_per_capita']]

def porMes(a, m, d):
    df = d.copy()
    df3 = df[(df.Any == a) & (df.mes_n == m)]
    df3 =df3.groupby(['producto'])[['valor', 'precio', 'volumen', 'penetracion', 'volXprecio', 'consumo/gasto','gasto_per_capita','consumo_per_capita']].mean()
    df3['anyo'] = a
    df3['mes'] = NUM2MES[m]
    df3['mes_n'] = m
    df3['producto'] = df3.index
    return df3[['producto', 'anyo', 'mes','mes_n', 'valor', 'precio', 'volumen', 'penetracion', 'volXprecio', 'consumo/gasto','gasto_per_capita','consumo_per_capita']]

In [300]:
df1 = porAny(2018, df2)
df1.head()

,producto,any,valor,precio,volumen,penetracion,volXprecio,consumo/gasto,gasto_per_capita,consumo_per_capita
0,TOTAL PATATAS,2018,84640.08,0.78,108430.72,79.40,84575.9616,1.286486,1.85,2.38
1,PATATAS FRESCAS,2018,54688.29,0.69,79445.66,68.46,54817.5054,1.450000,1.20,1.74
2,PATATAS CONGELADAS,2018,4857.79,1.21,3999.90,12.06,4839.8790,0.818182,0.11,0.09
3,PATATAS PROCESADAS,2018,25094.00,5.02,4997.03,45.94,25085.0906,0.200000,0.55,0.11
4,T.HORTALIZAS FRESCAS,2018,376688.56,1.79,209957.24,97.27,375823.4596,0.557576,8.25,4.60


In [302]:
df3 = porAny(2019, df2)
df3.head()

,producto,any,valor,precio,volumen,penetracion,volXprecio,consumo/gasto,gasto_per_capita,consumo_per_capita
9990,TOTAL PATATAS,2019,98564.75,0.90,109465.69,80.01,98519.1210,1.107477,2.14,2.37
9991,PATATAS FRESCAS,2019,66698.49,0.83,79929.19,67.20,66341.2277,1.193103,1.45,1.73
9992,PATATAS CONGELADAS,2019,4855.67,1.26,3846.64,11.51,4846.7664,0.727273,0.11,0.08
9993,PATATAS PROCESADAS,2019,27010.59,5.26,5137.97,46.22,27025.7222,0.186441,0.59,0.11
9994,PATATAS FRITAS,2019,25518.21,6.21,4107.22,44.11,25505.8362,0.163636,0.55,0.09


In [303]:
df4 = porAny(2020, df2)
df4.head()

,producto,any,valor,precio,volumen,penetracion,volXprecio,consumo/gasto,gasto_per_capita,consumo_per_capita
21006,TOTAL PATATAS,2020,97965.62,0.96,101985.34,79.37,97905.9264,1.040724,2.21,2.30
21007,PATATAS FRESCAS,2020,64713.94,0.90,71942.24,66.50,64748.0160,1.109589,1.46,1.62
21008,PATATAS CONGELADAS,2020,5190.90,1.31,3948.14,11.44,5172.0634,0.750000,0.12,0.09
21009,PATATAS PROCESADAS,2020,28060.78,5.38,5218.99,46.63,28078.1662,0.190476,0.63,0.12
21010,PATATAS FRITAS,2020,26597.58,6.21,4282.27,44.59,26592.8967,0.166667,0.60,0.10


In [304]:
productos = pd.concat([df1,df3,df4],axis= 0)
productos

,producto,any,valor,precio,volumen,penetracion,volXprecio,consumo/gasto,gasto_per_capita,consumo_per_capita
0,TOTAL PATATAS,2018,84640.08,0.78,108430.72,79.40,84575.9616,1.286486,1.85,2.38
1,PATATAS FRESCAS,2018,54688.29,0.69,79445.66,68.46,54817.5054,1.450000,1.20,1.74
2,PATATAS CONGELADAS,2018,4857.79,1.21,3999.90,12.06,4839.8790,0.818182,0.11,0.09
3,PATATAS PROCESADAS,2018,25094.00,5.02,4997.03,45.94,25085.0906,0.200000,0.55,0.11
4,T.HORTALIZAS FRESCAS,2018,376688.56,1.79,209957.24,97.27,375823.4596,0.557576,8.25,4.60
...,...,...,...,...,...,...,...,...,...,...
26629,UVAS,2020,42755.62,2.69,15918.49,NaN,42820.7381,0.375000,0.96,0.36
26630,KIWI,2020,41949.06,3.42,12253.15,NaN,41905.7730,0.294737,0.95,0.28
26631,PATATAS FRESCAS,2020,76899.34,0.86,89679.98,NaN,77124.7828,1.167630,1.73,2.02
26632,T.HORTALIZAS FRESCAS,2020,472592.86,1.98,238554.47,NaN,472337.8506,0.504690,10.66,5.38


<h2> Que tal fueron las patatas de precio?

In [257]:
productos.prods.unique()

array(['AGUACATE', 'AJOS', 'ALBARICOQUES', 'ALCACHOFAS', 'BERENJENAS',
       'BROCOLI', 'CALABACINES', 'CEBOLLAS', 'CEREZAS',
       'CHAMPIÑONES+O.SETAS', 'CHIRIMOYA', 'CIRUELAS', 'COLES',
       'ESPARRAGOS', 'FRESAS/FRESON', 'FRUTAS IV GAMA', 'JUDIAS VERDES',
       'KIWI', 'LECHUGA/ESC./ENDIVIA', 'LIMONES', 'MANDARINAS',
       'MANZANAS', 'MELOCOTONES', 'MELON', 'NARANJAS', 'NECTARINAS',
       'OTR.HORTALIZAS/VERD.', 'OTRAS FRUTAS FRESCAS',
       'PATATAS CONGELADAS', 'PATATAS FRESCAS', 'PATATAS FRITAS',
       'PATATAS PROCESADAS', 'PEPINOS', 'PERAS', 'PIMIENTOS', 'PIÑA',
       'PLATANOS', 'POMELO', 'SANDIA', 'T.FRUTAS FRESCAS',
       'T.HORTALIZAS FRESCAS', 'TOMATES', 'TOTAL PATATAS', 'UVAS',
       'VERD./HORT. IV GAMA', 'VERDURAS DE HOJA', 'ZANAHORIAS', 'APIO',
       'COLIFLOR', 'MANGO', 'PUERRO'], dtype=object)

In [ ]:
productos['prods'] = productos.index
productos[productos.prods == 'TOTAL PATATAS']

In [258]:
productos['prods'] = productos.index
productos[productos.prods == 'T.HORTALIZAS FRESCAS']

,valor,precio,volumen,penetracion,volXprecio,consumo/gasto,gasto_per_capita,consumo_per_capita,any,prods
producto,,,,,,,,,,
T.HORTALIZAS FRESCAS,43326.933241,1.815926,24111.204954,95.811852,43327.216731,0.553194,8.499306,4.682407,2018,T.HORTALIZAS FRESCAS
T.HORTALIZAS FRESCAS,45681.168102,1.896759,24317.932454,95.796898,45687.543270,0.528975,8.955602,4.724769,2019,T.HORTALIZAS FRESCAS
T.HORTALIZAS FRESCAS,73613.730531,1.962389,37806.478673,97.213426,73653.677066,0.510968,11.123363,5.642832,2020,T.HORTALIZAS FRESCAS


In [259]:
productos['prods'] = productos.index
productos[productos.prods == 'LIMONES']

,valor,precio,volumen,penetracion,volXprecio,consumo/gasto,gasto_per_capita,consumo_per_capita,any,prods
producto,,,,,,,,,,
LIMONES,1800.738287,1.784352,1006.348380,36.179583,1800.656149,0.574976,0.363750,0.203935,2018,LIMONES
LIMONES,1529.556852,1.567130,978.382037,35.223380,1529.459009,0.649482,0.302454,0.193889,2019,LIMONES
LIMONES,2894.755929,1.759292,1607.859646,38.542778,2894.868024,0.577074,0.449381,0.252566,2020,LIMONES


<h3> Comparando los productos mas caros agrupando por productos por meses

In [334]:
df2 = df
df2['volXprecio'] = df2.apply(lambda x: x.volumen * x.precio, axis= 1)
df2['consumo/gasto'] = df2.apply(lambda x: x.consumo_per_capita / x.gasto_per_capita if x.gasto_per_capita else 0, axis= 1)
prods_por_meses = pd.DataFrame(columns = ['producto', 'anyo', 'mes','mes_n', 'valor', 'precio', 'volumen', 'penetracion', 'volXprecio', 
                                          'consumo/gasto','gasto_per_capita','consumo_per_capita'])
for y, m in sorted(set(zip(df.Any, df.mes_n))):
    prods_por_meses = pd.concat([prods_por_meses, porMes(y,m,df)], axis= 0)
prods_por_meses = prods_por_meses.reset_index(drop= True) 

In [339]:
# porMes(2018, 1,df2)
prods_por_meses.round(2)

,producto,anyo,mes,mes_n,valor,precio,volumen,penetracion,volXprecio,consumo/gasto,gasto_per_capita,consumo_per_capita
0,AGUACATE,2018,Enero,1,1629.13,3.44,486.55,24.97,1629.04,0.30,0.35,0.10
1,AJOS,2018,Enero,1,1192.95,4.22,285.85,33.21,1192.74,0.24,0.23,0.06
2,ALBARICOQUES,2018,Enero,1,27.58,1.26,15.09,0.56,27.62,0.11,0.00,0.00
3,ALCACHOFAS,2018,Enero,1,1713.33,2.00,940.39,21.44,1712.26,0.51,0.26,0.14
4,BERENJENAS,2018,Enero,1,852.86,1.78,487.04,22.84,852.77,0.57,0.14,0.08
...,...,...,...,...,...,...,...,...,...,...,...,...
1588,SANDIA,2020,Noviembre,11,860.28,1.35,637.46,NaN,860.57,0.50,0.02,0.01
1589,T.FRUTAS FRESCAS,2020,Noviembre,11,597092.66,1.75,341351.65,NaN,597365.39,0.57,13.47,7.70
1590,T.HORTALIZAS FRESCAS,2020,Noviembre,11,472592.86,1.98,238554.47,NaN,472337.85,0.50,10.66,5.38
1591,TOMATES,2020,Noviembre,11,87551.48,1.80,48644.85,NaN,87560.73,0.56,1.97,1.10


<h2>1- Cuales son los productos mas caros por orden de meses?

In [365]:
for y, m in sorted(set(zip(prods_por_meses.anyo, prods_por_meses.mes_n))):
    print(f'{y:>50} / {NUM2MES[m]}')
    data = prods_por_meses[(prods_por_meses.anyo == y) & (prods_por_meses.mes_n == m) & #(df.CCAA != 'Total Nacional') &
             (prods_por_meses.producto != 'T.FRUTAS FRESCAS') & (prods_por_meses.producto != 'T.HORTALIZAS FRESCAS') &
             (prods_por_meses.producto != 'TOTAL PATATAS')].sort_values(by= 'precio', ascending= False).head().round(3)
    t = data.keys()
    print(f'{t[0]:^20}|{t[1]:^6}|{t[3]:^5}|{t[4]:^10}|{t[7]:^14}|{t[6]:^10}|{t[5]:^10}|{t[8]:^10}|{t[9]:^13}|Gas/Cap|Cons/Cap\n{"-"*120}')
    for i in range(min(5, data.mes.shape[0])):
        r = data.iloc[i]
        print(f'{r[0]:^20}|{r[1]:^6}|{r[3]:^5}|{r[4]:^10}|{r[7]:^14}|{r[6]:^10}|{r[5]:^10}|{r[8]:^10}|{r[9]:^13}|{r[10]:^7}|{r[11]:^10}')
    print()

                                              2018 / Enero
      producto      | anyo |mes_n|  valor   | penetracion  | volumen  |  precio  |volXprecio|consumo/gasto|Gas/Cap|Cons/Cap
------------------------------------------------------------------------------------------------------------------------
 PATATAS PROCESADAS | 2018 |  1  | 2788.222 |    42.447    | 555.226  |  5.051   | 2787.619 |    0.201    | 0.477 |  0.094   
      CEREZAS       | 2018 |  1  | 117.105  |    1.418     |  23.306  |  4.964   | 117.033  |    0.076    | 0.021 |  0.003   
        AJOS        | 2018 |  1  | 1192.948 |    33.211    | 285.849  |  4.216   | 1192.737 |    0.24     | 0.234 |  0.056   
CHAMPIÑONES+O.SETAS | 2018 |  1  | 2328.185 |    42.161    | 639.517  |  3.685   | 2328.505 |    0.274    | 0.444 |  0.121   
   FRESAS/FRESON    | 2018 |  1  | 3158.398 |    31.962    | 896.518  |  3.546   | 3157.285 |    0.287    | 0.616 |  0.176   

                                              2018 / Febrero
    

<h2> 2- Cuales son los productos de más volumen por orden de meses?

In [366]:
for y, m in sorted(set(zip(prods_por_meses.anyo, prods_por_meses.mes_n))):
    print(f'{y:>50} / {NUM2MES[m]}')
    data = prods_por_meses[(prods_por_meses.anyo == y) & (prods_por_meses.mes_n == m) & #(df.CCAA != 'Total Nacional') &
             (prods_por_meses.producto != 'T.FRUTAS FRESCAS') & (prods_por_meses.producto != 'T.HORTALIZAS FRESCAS') &
             (prods_por_meses.producto != 'TOTAL PATATAS')].sort_values(by= 'volumen', ascending= False).head().round(3)
    t = data.keys()
    print(f'{t[0]:^20}|{t[1]:^6}|{t[3]:^5}|{t[4]:^10}|{t[7]:^14}|{t[6]:^10}|{t[5]:^10}|{t[8]:^10}|{t[9]:^13}|Gas/Cap|Cons/Cap\n{"-"*120}')
    for i in range(min(5, data.mes.shape[0])):
        r = data.iloc[i]
        print(f'{r[0]:^20}|{r[1]:^6}|{r[3]:^5}|{r[4]:^10}|{r[7]:^14}|{r[6]:^10}|{r[5]:^10}|{r[8]:^10}|{r[9]:^13}|{r[10]:^7}|{r[11]:^10}')
    print()

                                              2018 / Enero
      producto      | anyo |mes_n|  valor   | penetracion  | volumen  |  precio  |volXprecio|consumo/gasto|Gas/Cap|Cons/Cap
------------------------------------------------------------------------------------------------------------------------
      NARANJAS      | 2018 |  1  | 9806.451 |    60.809    |10712.767 |  0.914   | 9829.012 |    1.098    | 1.977 |  2.152   
  PATATAS FRESCAS   | 2018 |  1  | 6076.478 |    64.574    | 8827.296 |  0.677   | 6075.166 |    1.487    | 1.134 |  1.656   
      PLATANOS      | 2018 |  1  | 8331.846 |    80.58     | 5757.692 |  1.452   | 8338.373 |    0.692    | 1.678 |  1.153   
OTR.HORTALIZAS/VERD.| 2018 |  1  |10735.201 |    79.081    | 5560.121 |  1.968   |10731.538 |    0.512    | 1.979 |  1.013   
     MANDARINAS     | 2018 |  1  | 7054.201 |    63.371    | 5206.387 |  1.344   | 7042.817 |    0.748    | 1.363 |  1.011   

                                              2018 / Febrero
    

      TOMATES       | 2020 |  7  | 120124.2 |     nan      | 74111.99 |   1.62   |120061.424|    0.616    | 2.71  |   1.67   
      PLATANOS      | 2020 |  7  | 67823.14 |     nan      | 42572.49 |   1.59   |67690.259 |    0.627    | 1.53  |   0.96   

                                              2020 / Agosto
      producto      | anyo |mes_n|  valor   | penetracion  | volumen  |  precio  |volXprecio|consumo/gasto|Gas/Cap|Cons/Cap
------------------------------------------------------------------------------------------------------------------------
       SANDIA       | 2020 |  8  | 66535.96 |     nan      | 99670.37 |   0.67   |66779.148 |     1.5     |  1.5  |   2.25   
       MELON        | 2020 |  8  | 80985.69 |     nan      | 81329.97 |   1.0    | 81329.97 |     1.0     | 1.83  |   1.83   
  PATATAS FRESCAS   | 2020 |  8  | 66028.82 |     nan      | 74085.89 |   0.89   |65936.442 |    1.121    | 1.49  |   1.67   
      TOMATES       | 2020 |  8  | 105649.3 |     nan      | 644

<h2>3- De que es el mayor consumo per capita por orden de meses?

In [367]:
for y, m in sorted(set(zip(prods_por_meses.anyo, prods_por_meses.mes_n))):
    print(f'{y:>50} / {NUM2MES[m]}')
    data = prods_por_meses[(prods_por_meses.anyo == y) & (prods_por_meses.mes_n == m) & #(df.CCAA != 'Total Nacional') &
             (prods_por_meses.producto != 'T.FRUTAS FRESCAS') & (prods_por_meses.producto != 'T.HORTALIZAS FRESCAS') &
             (prods_por_meses.producto != 'TOTAL PATATAS')].sort_values(by= 'consumo_per_capita', ascending= False).head().round(3)
    t = data.keys()
    print(f'{t[0]:^20}|{t[1]:^6}|{t[3]:^5}|{t[4]:^10}|{t[7]:^14}|{t[6]:^10}|{t[5]:^10}|{t[8]:^10}|{t[9]:^13}|Gas/Cap|Cons/Cap\n{"-"*120}')
    for i in range(min(5, data.mes.shape[0])):
        r = data.iloc[i]
        print(f'{r[0]:^20}|{r[1]:^6}|{r[3]:^5}|{r[4]:^10}|{r[7]:^14}|{r[6]:^10}|{r[5]:^10}|{r[8]:^10}|{r[9]:^13}|{r[10]:^7}|{r[11]:^10}')
    print()

                                              2018 / Enero
      producto      | anyo |mes_n|  valor   | penetracion  | volumen  |  precio  |volXprecio|consumo/gasto|Gas/Cap|Cons/Cap
------------------------------------------------------------------------------------------------------------------------
      NARANJAS      | 2018 |  1  | 9806.451 |    60.809    |10712.767 |  0.914   | 9829.012 |    1.098    | 1.977 |  2.152   
  PATATAS FRESCAS   | 2018 |  1  | 6076.478 |    64.574    | 8827.296 |  0.677   | 6075.166 |    1.487    | 1.134 |  1.656   
      PLATANOS      | 2018 |  1  | 8331.846 |    80.58     | 5757.692 |  1.452   | 8338.373 |    0.692    | 1.678 |  1.153   
OTR.HORTALIZAS/VERD.| 2018 |  1  |10735.201 |    79.081    | 5560.121 |  1.968   |10731.538 |    0.512    | 1.979 |  1.013   
     MANDARINAS     | 2018 |  1  | 7054.201 |    63.371    | 5206.387 |  1.344   | 7042.817 |    0.748    | 1.363 |  1.011   

                                              2018 / Febrero
    

      producto      | anyo |mes_n|  valor   | penetracion  | volumen  |  precio  |volXprecio|consumo/gasto|Gas/Cap|Cons/Cap
------------------------------------------------------------------------------------------------------------------------
  PATATAS FRESCAS   | 2020 |  9  | 75146.3  |     nan      | 84453.33 |   0.89   |75163.464 |    1.118    |  1.7  |   1.9    
       MELON        | 2020 |  9  | 72800.62 |     nan      | 77570.51 |   0.94   |72916.279 |    1.067    | 1.64  |   1.75   
      TOMATES       | 2020 |  9  | 115161.0 |     nan      | 64894.57 |   1.77   |114863.389|    0.562    |  2.6  |   1.46   
       SANDIA       | 2020 |  9  | 45308.76 |     nan      | 51830.72 |   0.87   |45092.726 |    1.147    | 1.02  |   1.17   
      PLATANOS      | 2020 |  9  | 73126.22 |     nan      | 44651.93 |   1.64   |73229.165 |    0.612    | 1.65  |   1.01   

                                              2020 / Octubre
      producto      | anyo |mes_n|  valor   | penetracion  | vo

<h2>4- Cuales es el mayor gasto per capita?

In [368]:
for y, m in sorted(set(zip(prods_por_meses.anyo, prods_por_meses.mes_n))):
    print(f'{y:>50} / {NUM2MES[m]}')
    data = prods_por_meses[(prods_por_meses.anyo == y) & (prods_por_meses.mes_n == m) & #(df.CCAA != 'Total Nacional') &
             (prods_por_meses.producto != 'T.FRUTAS FRESCAS') & (prods_por_meses.producto != 'T.HORTALIZAS FRESCAS') &
             (prods_por_meses.producto != 'TOTAL PATATAS')].sort_values(by= 'gasto_per_capita', ascending= False).head().round(3)
    t = data.keys()
    print(f'{t[0]:^20}|{t[1]:^6}|{t[3]:^5}|{t[4]:^10}|{t[7]:^14}|{t[6]:^10}|{t[5]:^10}|{t[8]:^10}|{t[9]:^13}|Gas/Cap|Cons/Cap\n{"-"*120}')
    for i in range(min(5, data.mes.shape[0])):
        r = data.iloc[i]
        print(f'{r[0]:^20}|{r[1]:^6}|{r[3]:^5}|{r[4]:^10}|{r[7]:^14}|{r[6]:^10}|{r[5]:^10}|{r[8]:^10}|{r[9]:^13}|{r[10]:^7}|{r[11]:^10}')
    print()

                                              2018 / Enero
      producto      | anyo |mes_n|  valor   | penetracion  | volumen  |  precio  |volXprecio|consumo/gasto|Gas/Cap|Cons/Cap
------------------------------------------------------------------------------------------------------------------------
OTR.HORTALIZAS/VERD.| 2018 |  1  |10735.201 |    79.081    | 5560.121 |  1.968   |10731.538 |    0.512    | 1.979 |  1.013   
      NARANJAS      | 2018 |  1  | 9806.451 |    60.809    |10712.767 |  0.914   | 9829.012 |    1.098    | 1.977 |  2.152   
      PLATANOS      | 2018 |  1  | 8331.846 |    80.58     | 5757.692 |  1.452   | 8338.373 |    0.692    | 1.678 |  1.153   
     MANDARINAS     | 2018 |  1  | 7054.201 |    63.371    | 5206.387 |  1.344   | 7042.817 |    0.748    | 1.363 |  1.011   
      TOMATES       | 2018 |  1  | 7501.97  |    74.203    | 4437.149 |  1.709   | 7498.006 |    0.589    | 1.317 |  0.774   

                                              2018 / Febrero
    

      producto      | anyo |mes_n|  valor   | penetracion  | volumen  |  precio  |volXprecio|consumo/gasto|Gas/Cap|Cons/Cap
------------------------------------------------------------------------------------------------------------------------
      TOMATES       | 2020 | 10  | 96244.33 |     nan      | 52238.0  |   1.84   | 96117.92 |    0.544    | 2.17  |   1.18   
      PLATANOS      | 2020 | 10  | 93993.44 |     nan      | 55264.94 |   1.7    |93950.398 |    0.59     | 2.12  |   1.25   
  PATATAS FRESCAS   | 2020 | 10  | 83644.14 |     nan      | 95916.0  |   0.87   | 83446.92 |    1.143    | 1.89  |   2.16   
      MANZANAS      | 2020 | 10  | 80253.44 |     nan      | 51560.58 |   1.56   |80434.505 |    0.641    | 1.81  |   1.16   
      NARANJAS      | 2020 | 10  | 64232.3  |     nan      | 39240.26 |   1.64   |64354.026 |    0.607    | 1.45  |   0.88   

                                              2020 / Noviembre
      producto      | anyo |mes_n|  valor   | penetracion  | 

<h2> 5- Relaciones Volumen-Precio

In [393]:
for y, m in sorted(set(zip(prods_por_meses.anyo, prods_por_meses.mes_n))):
    print(f'{y:>50} / {NUM2MES[m]}')
    data = prods_por_meses[(prods_por_meses.anyo == y) & (prods_por_meses.mes_n == m) & #(df.CCAA != 'Total Nacional') &
             (prods_por_meses.producto != 'T.FRUTAS FRESCAS') & (prods_por_meses.producto != 'T.HORTALIZAS FRESCAS') &
             (prods_por_meses.producto != 'TOTAL PATATAS')
                          ].sort_values(by= 'volXprecio', ascending= False).head().round(3)
    t = data.keys()
    print(f'{t[0]:^20}|{t[1]:^6}|{t[3]:^5}|{t[4]:^10}|{t[7]:^14}|{t[6]:^10}|{t[5]:^10}|{t[8]:^10}|{t[9]:^13}|Gas/Cap|Cons/Cap\n{"-"*120}')
    for i in range(min(5, data.mes.shape[0])):
        r = data.iloc[i]
        print(f'{r[0]:^20}|{r[1]:^6}|{r[3]:^5}|{r[4]:^10}|{r[7]:^14}|{r[6]:^10}|{r[5]:^10}|{r[8]:^10}|{r[9]:^13}|{r[10]:^7}|{r[11]:^10}')
    print()

                                              2018 / Enero
      producto      | anyo |mes_n|  valor   | penetracion  | volumen  |  precio  |volXprecio|consumo/gasto|Gas/Cap|Cons/Cap
------------------------------------------------------------------------------------------------------------------------
OTR.HORTALIZAS/VERD.| 2018 |  1  |10735.201 |    79.081    | 5560.121 |  1.968   |10731.538 |    0.512    | 1.979 |  1.013   
      NARANJAS      | 2018 |  1  | 9806.451 |    60.809    |10712.767 |  0.914   | 9829.012 |    1.098    | 1.977 |  2.152   
      PLATANOS      | 2018 |  1  | 8331.846 |    80.58     | 5757.692 |  1.452   | 8338.373 |    0.692    | 1.678 |  1.153   
      TOMATES       | 2018 |  1  | 7501.97  |    74.203    | 4437.149 |  1.709   | 7498.006 |    0.589    | 1.317 |  0.774   
     MANDARINAS     | 2018 |  1  | 7054.201 |    63.371    | 5206.387 |  1.344   | 7042.817 |    0.748    | 1.363 |  1.011   

                                              2018 / Febrero
    

                                              2020 / Agosto
      producto      | anyo |mes_n|  valor   | penetracion  | volumen  |  precio  |volXprecio|consumo/gasto|Gas/Cap|Cons/Cap
------------------------------------------------------------------------------------------------------------------------
      TOMATES       | 2020 |  8  | 105649.3 |     nan      | 64452.71 |   1.64   |105702.444|    0.609    | 2.38  |   1.45   
       MELON        | 2020 |  8  | 80985.69 |     nan      | 81329.97 |   1.0    | 81329.97 |     1.0     | 1.83  |   1.83   
       SANDIA       | 2020 |  8  | 66535.96 |     nan      | 99670.37 |   0.67   |66779.148 |     1.5     |  1.5  |   2.25   
  PATATAS FRESCAS   | 2020 |  8  | 66028.82 |     nan      | 74085.89 |   0.89   |65936.442 |    1.121    | 1.49  |   1.67   
    MELOCOTONES     | 2020 |  8  | 59874.19 |     nan      | 31865.5  |   1.88   | 59907.14 |    0.533    | 1.35  |   0.72   

                                              2020 / Septiembre


In [389]:
"""
0- Aburrire con el mapita
1- Cuales son los productos mas caros por orden de meses?
2- Cuales son los productos de más volumen por orden de meses?
3- De que es el mayor consumo per capita por orden de meses?
4- Cuales es el mayor gasto per capita?
5- Relaciones Volumen-Precio 

""" 

'\n0- Aburrire con el mapita\n1- Cuales son los productos mas caros por orden de meses?\n2- Cuales son los productos de más volumen por orden de meses?\n3- De que es el mayor consumo per capita por orden de meses?\n4- Cuales es el mayor gasto per capita?\n5- Relaciones Volumen-Precio \n\n'

<h2> Progreso de las stats por cada producto

In [400]:
import plotly.express as px

p = prods_por_meses[(prods_por_meses.producto == 'T.FRUTAS FRESCAS') | (prods_por_meses.producto == 'T.HORTALIZAS FRESCAS') | (prods_por_meses.producto == 'TOTAL PATATAS')]
p
fig = px.pie(p[p.anyo==2018], values='volumen', names='producto', title='Volumen por Producto 2018')
fig.show()
fig = px.pie(p[p.anyo==2019], values='volumen', names='producto', title='Volumen por Producto 2019')
fig.show()
fig = px.pie(p[p.anyo==2020], values='volumen', names='producto', title='Volumen por Producto 2020')
fig.show()

In [401]:
fig = px.pie(p[p.anyo==2018], values='penetracion', names='producto', title='Volumen por Producto 2018')
fig.show()
fig = px.pie(p[p.anyo==2019], values='penetracion', names='producto', title='Volumen por Producto 2019')
fig.show()
fig = px.pie(p[p.anyo==2020], values='penetracion', names='producto', title='Volumen por Producto 2020')
fig.show()

In [402]:
fig = px.pie(p[p.anyo==2018], values='consumo_per_capita', names='producto', title='Volumen por Producto 2018')
fig.show()
fig = px.pie(p[p.anyo==2019], values='consumo_per_capita', names='producto', title='Volumen por Producto 2019')
fig.show()
fig = px.pie(p[p.anyo==2020], values='consumo_per_capita', names='producto', title='Volumen por Producto 2020')
fig.show()

In [408]:
prods_por_meses['time'] = prods_por_meses.apply(lambda r: str(r.anyo)+'-'+r.mes, axis= 1)
prods_por_meses.head(2)

,producto,anyo,mes,mes_n,valor,precio,volumen,penetracion,volXprecio,consumo/gasto,gasto_per_capita,consumo_per_capita,time
0,AGUACATE,2018,Enero,1,1629.130000,3.438333,486.546667,24.971111,1629.039356,0.302864,0.350556,0.103889,2018-Enero
1,AJOS,2018,Enero,1,1192.948333,4.215556,285.848889,33.210556,1192.737478,0.240279,0.234444,0.056111,2018-Enero


In [427]:
f = prods_por_meses[(prods_por_meses.producto != 'T.FRUTAS FRESCAS') & (prods_por_meses.producto != 'T.HORTALIZAS FRESCAS') & (prods_por_meses.producto != 'TOTAL PATATAS')]
fig = px.scatter(f, x="volumen", y="precio", animation_frame="time", animation_group="producto",range_y=[0, f.precio.max()],range_x=[0, f.volumen.max()],
           size="volumen", color="valor", hover_name="producto")

fig.show()